In [23]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

dist = pd.read_csv('../data/times v4.csv')
le = sklearn.preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])

num_vehicles = 20

config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}



def print_solution(data, manager, routing, solution):
    visited = [0 for i in range(cnt_terminals)]
    paths = []
    for vehicle in range(num_vehicles):
        path = []
        i = routing.Start(vehicle)
        while not routing.IsEnd(i):
            i = solution.Value(routing.NextVar(i))
            if i > 0 and i <= cnt_terminals:
                visited[i - 1] = 1
                path.append(i - 1)
        paths.append(path)
    return visited, paths

def origin(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            plan_output += f"({routing.GetArcCostForVehicle(previous_index, index, vehicle_id) / 100})"
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

inf = int(1e4)
cnt_terminals = config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)

penalties = [1 for i in range(cnt_terminals + 2)]
penalties[0] = 0
penalties[-1] = 0

distance_matrix = (distance_matrix * 100).astype(int)
inf *= 100
config['work_time'] *= 100

In [2]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': num_vehicles,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']

In [3]:
manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                   vrp_data['num_vehicles'], vrp_data['starts'], vrp_data['ends'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Define cost of each arc.

def distance_callback(from_index, to_index):
    """Returns the manhattan distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return vrp_data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    config['work_time'],  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Allow to drop nodes.
penalty = inf
for node in range(1, len(vrp_data['distance_matrix']) - 1):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty * (100 if node < 20 else 1))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.time_limit.seconds = 100

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
    
visited, paths = print_solution(vrp_data, manager, routing, solution)

In [27]:
path = paths[0]

dist = 0
for i in range(len(path) - 1):
    v = path[i]
    u = path[i + 1]
    print(distance_matrix[u, v] / 100, end=" ")
    dist += distance_matrix[u, v]

64.0 141.0 120.0 32.0 20.0 36.0 22.0 52.0 29.0 56.0 46.0 112.0 115.0 54.0 73.0 45.0 114.0 137.0 84.0 51.0 43.0 51.0 25.0 21.0 56.0 68.0 36.0 46.0 41.0 65.0 51.0 40.0 42.0 48.0 13.0 19.0 39.0 119.0 121.0 70.0 85.0 106.0 75.0 48.0 69.0 

In [24]:
origin(vrp_data, manager, routing, solution)

Objective: 952174400
Route for vehicle 0:
 0 -> (10.0) 1602 -> (11.0) 1121 -> (13.0) 1014 -> (12.0) 649 -> (12.0) 66 -> (13.0) 47 -> (13.0) 1504 -> (14.0) 590 -> (15.0) 1513 -> (15.0) 922 -> (11.0) 1539 -> (10.0) 619 -> (12.0) 887 -> (15.0) 528 -> (10.0) 153 -> (15.0) 1185 -> (22.0) 1526 -> (13.0) 294 -> (13.0) 1145 -> (12.0) 35 -> (11.0) 892 -> (11.0) 1304 -> (11.0) 40 -> (13.0) 539 -> (14.0) 602 -> (11.0) 231 -> (12.0) 22 -> (10.0) 1471 -> (13.0) 626 -> (11.0) 18 -> (14.0) 818 -> (13.0) 53 -> (14.0) 407 -> (15.0) 39 -> (14.0) 591 -> (11.0) 48 -> (18.0) 263 -> (21.0) 861 -> (10.0) 1473 -> (11.0) 151 -> (12.0) 531 -> (11.0) 1366 -> (11.0) 1552 -> (13.0) 795 -> (14.0) 1170 -> (10.0) 1557 -> (0.0)1631
Distance of the route: 59000m

Route for vehicle 1:
 0 -> (10.0) 1603 -> (12.0) 1141 -> (11.0) 278 -> (12.0) 300 -> (11.0) 1139 -> (11.0) 934 -> (11.0) 628 -> (13.0) 290 -> (11.0) 1062 -> (10.0) 900 -> (12.0) 647 -> (10.0) 847 -> (11.0) 1502 -> (10.0) 1464 -> (11.0) 576 -> (10.0) 1351 -> (1

In [ ]:
print(path)

In [13]:
path = paths[1]

In [14]:
print(path)

[1602, 1140, 277, 299, 1138, 933, 627, 289, 1061, 899, 646, 846, 1501, 1463, 575, 1350, 1456, 1534, 909, 288, 305, 285, 1182, 1446, 849, 1324, 1102, 597, 1355, 91, 491, 1300, 827, 1118, 1170, 537, 332, 1570, 1357, 1176, 893, 633, 815, 284, 839, 309, 1356, 1062, 489, 328, 939]


In [15]:

dist = 0
for i in range(len(path) - 1):
    v = path[i]
    u = path[i + 1]
    dist += distance_matrix[u, v]

In [16]:
dist / 100

3506.0